<a href="https://www.kaggle.com/code/samuelvangorden/multiple-choice-transformer?scriptVersionId=154736576" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Code

In [1]:
import numpy as np
import pandas as pd
import torch
import gc

from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForMultipleChoice, TrainingArguments, Trainer

from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Load/Preprocess Data

In [2]:
MODEL_DIR = "/kaggle/input/huggingface-bert/"

train = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/train.csv")
mapping = {k: v for v, k in enumerate(np.sort(train['answer'].unique()))}
train['label'] = train['answer'].map(mapping)
train = Dataset.from_pandas(train, split='train')
train, val = train.train_test_split(test_size=0.1).values()
data_dict = DatasetDict({"train":train, "val":val})

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


## Load/Build LLM Model

In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR + "bert-base-uncased")
choices = ["A", "B", "C", "D", "E"]

def preprocess_function(examples, return_tensors=None, padding=False, truncation=True):
    questions = [[ex]*5 for ex in examples["prompt"]]
    answers = [examples[c] for c in choices]
    answers = np.array(answers)
    answers = answers.reshape(answers.shape[1], answers.shape[0]).tolist()

    questions = sum(questions, [])
    answers = sum(answers, [])

    tokenized_examples = tokenizer(text=questions, text_pair=answers, return_tensors=return_tensors, padding=padding, truncation=truncation)
    return {k: [v[i : i + 5] for i in range(0, len(v), 5)] for k, v in tokenized_examples.items()}

tokenized_data = data_dict.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/1 [00:00<?, ?ba/s]

In [4]:
@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [5]:
model = AutoModelForMultipleChoice.from_pretrained(MODEL_DIR + "bert-base-uncased")

training_args = TrainingArguments(
    output_dir="bert_fine_tuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=False,
    report_to="none",
    disable_tqdm=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["val"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
)

trainer.train()
trainer.save_model()
del trainer, model, train, val, data_dict, tokenizer
gc.collect()

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at /kaggle/input/huggingface-bert/bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,1.589902


230

In [6]:
test = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")
test = Dataset.from_pandas(test, split='test')
inv_map = {0:"A", 1:"B", 2:"C", 3:"D", 4:"E"}
choices = list(inv_map.values())
submission = pd.DataFrame()
submission["id"] = test["id"]

tokenizer = AutoTokenizer.from_pretrained("bert_fine_tuned")
questions = [[ex]*5 for ex in test["prompt"]]
answers = [test[c] for c in choices]
answers = np.array(answers)
answers = answers.reshape(answers.shape[1], answers.shape[0]).tolist()
questions = sum(questions, [])
answers = sum(answers, [])

inputs = tokenizer(text=questions, text_pair=answers, return_tensors="pt", padding=True)
model = AutoModelForMultipleChoice.from_pretrained("bert_fine_tuned")
model.eval()

with torch.no_grad():
    outputs = model(**{k: v.unsqueeze(0) for k, v in inputs.items()})

del model, test, tokenizer, questions, answers
gc.collect()

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


7

In [7]:
indices = [np.argsort(preds)[-3:] for preds in outputs.logits.detach().numpy().reshape((200,5))]
pred_answers = [" ".join([inv_map[pred] for pred in preds]) for preds in indices]
submission["prediction"] = pred_answers
submission.to_csv("submission.csv")